In [41]:
import numpy as np
X = {}
y = {}
X["train"] = np.load("./train_feature_save.npy",allow_pickle=True )
y["train"] = np.load("./train_label_save.npy")
X["dev"] = np.load("./test_feature_save.npy")
y["dev"] = np.load("./test_label_save.npy")
X["train"].shape, X["dev"].shape

((18000, 768), (1000, 768))

In [42]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_tweet'

In [43]:
y["train"]=y["train"].squeeze()
y["dev"]=y["dev"].squeeze()
y["train"].shape, y["dev"].shape

((18000,), (1000,))

In [44]:
from sklearn.linear_model import LogisticRegression
l2 = 10
model = LogisticRegression(penalty='l2', C=1/l2)

model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
model.score(X["dev"], y["dev"])

0.758

In [45]:
import sklearn.metrics as mc
predl = model.predict(X["dev"])
mc.f1_score(y["dev"], predl)

0.7618110236220473

In [46]:
fpr, tpr, thresholds = mc.roc_curve(y["dev"], pred)
mc.auc(fpr, tpr)

0.8427762776277629

In [47]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((18000, 769), (1000, 769))

In [48]:
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0]
np.all(np.linalg.eigvals(hessian) > 0)

True

In [49]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [50]:
np.all(np.linalg.eigvals(hessian) > 0)

True

In [51]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1000, 18000)

In [52]:
np.save("old_predictions.npy", pred)

In [53]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]

    X_k = np.delete(X["train"], top_k_index, axis=0)
    y_k = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    
    #print("prediction", prediction)
    

    return X_k, y_k, prediction

In [54]:
def new_train(k, dev_index, scores):
    X_k, y_k, prediction = Remove(k, scores, dev_index)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None, None, None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=1/l2)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model_k.predict_proba(test_point)[0][1]
    change = -(model.predict_proba(test_point)[0][1] - new)
    #change = model_k.predict_proba(test_point)[0][1]-model.predict_proba(test_point)[0][1]
    flip = (model.predict(test_point) == model_k.predict(test_point))
    
    """
    print("change    ", change)
    print("old       ", model.predict_proba(test_point)[0][1])
    print()
    """
    
  
    return change, flip, prediction,new

# exact_k

In [55]:
def exact_k(test_idx):
    for k in range(1, np.sum(y["train"])):
        _, flip,_ = new_train(k, test_idx, delta_pred)
        if flip == False:
            return k
    return None

In [56]:
"""
# Loop over all dev points:
import warnings
warnings.filterwarnings("ignore")

exact_ks = []
for test_idx in range(X["dev"].shape[0]):
    k =exact_k(test_idx)
    print(test_idx, k)
    exact_ks.append(k)

exact_ks=np.array(exact_ks)
np.save("exact_ks_IP.npy", exact_ks)
"""

'\n# Loop over all dev points:\nimport warnings\nwarnings.filterwarnings("ignore")\n\nexact_ks = []\nfor test_idx in range(X["dev"].shape[0]):\n    k =exact_k(test_idx)\n    print(test_idx, k)\n    exact_ks.append(k)\n\nexact_ks=np.array(exact_ks)\nnp.save("exact_ks_IP.npy", exact_ks)\n'

# Find approximate k by IF

In [57]:
def pred_change(k, scores, test_idx):
    X_k, y_k, prediction = Remove(k, delta_pred, test_idx)
    

In [58]:
def approximate_k(test_idx):
    old = pred[test_idx].item()
    
    if pred[test_idx] > 0.5:
        top_k_index = np.flip(delta_pred[test_idx].argsort())
    else:
        top_k_index = delta_pred[test_idx].argsort()
    
    for k in range(1, y["train"].shape[0]):
        change = -np.sum(delta_pred[test_idx][top_k_index[:k]])
        #print("prediction", change)
        
        if old > 0.5 and old + change < 0.5:
            return k
        elif old < 0.5 and old + change > 0.5:
            return k
        
    return None

In [59]:
# Loop over all dev points:
import warnings
warnings.filterwarnings("ignore")

appro_ks = []
new_predictions = []
for test_idx in range(X["dev"].shape[0]):
    appro_k = approximate_k(test_idx)
    if appro_k != None:
        _, _, _, new_prediction = new_train(appro_k, test_idx, delta_pred)
        print(test_idx, appro_k)
        print("old", pred[test_idx], "new", new_prediction)
        print()
        appro_ks.append(appro_k)
        new_predictions.append(new_prediction)
    else:
        appro_ks.append(None)
        new_predictions.append(None)



0 4172
old [0.05797461] new 0.9999557662539449

1 78
old [0.42315595] new 0.5121741929639311

2 465
old [0.18714946] new 0.6868215035171469

3 544
old [0.77461012] new 0.4105046117388845

4 468
old [0.72746982] new 0.4626776903112362

5 320
old [0.71942303] new 0.4594010605512647

6 368
old [0.75880423] new 0.39733008146784426

7 6
old [0.52058505] new 0.5325279458428768

8 107
old [0.36147057] new 0.4942426485690906

9 47
old [0.61132371] new 0.44835162333195533

10 962
old [0.18403992] new 0.7071791924360713

11 26
old [0.4595381] new 0.4812535294661075

12 8
old [0.51430261] new 0.49599156487820684

13 645
old [0.16953677] new 0.7252448443200603

15 27
old [0.4099405] new 0.4431869048658564

16 1053
old [0.80094078] new 0.33588442987563805

17 255
old [0.30243247] new 0.5497974266606519

18 271
old [0.73056958] new 0.4530208798851894

20 1040
old [0.82569726] new 0.22239962565633606

22 1912
old [0.13553859] new 0.9680154544922158

23 776
old [0.81688352] new 0.3373673757890958

24 

181 460
old [0.80414409] new 0.3194552247230138

182 2485
old [0.09694198] new 0.9884734176388081

183 173
old [0.33620914] new 0.5672488691780885

184 263
old [0.32768879] new 0.5528346929881837

185 81
old [0.39658957] new 0.5174888986349849

186 482
old [0.21253017] new 0.61134122944937

187 14
old [0.51850868] new 0.48984850033279953

188 114
old [0.40008227] new 0.49838612754231343

189 747
old [0.19057096] new 0.6573820924386168

190 97
old [0.61174338] new 0.5014339018708084

191 532
old [0.20479367] new 0.6532600832735392

192 1982
old [0.115554] new 0.9699210262909328

193 1768
old [0.11899616] new 0.9273408690303443

194 2
old [0.50259367] new 0.5023992639214848

195 14
old [0.46534177] new 0.480053756559152

196 59
old [0.56818558] new 0.5072561183400254

197 1654
old [0.10365803] new 0.9691105300907552

198 1258
old [0.90502893] new 0.035168783060873475

199 291
old [0.27304745] new 0.5264943940947449

200 976
old [0.88873005] new 0.05772061561127157

201 223
old [0.6848333

365 683
old [0.20382643] new 0.6816469599336495

366 37
old [0.4038846] new 0.53938230023301

367 549
old [0.79886276] new 0.35211099177086996

368 268
old [0.3111913] new 0.5390520895959421

369 275
old [0.67382366] new 0.4543262524954698

370 95
old [0.40255081] new 0.5031067119213164

371 2335
old [0.89161438] new 0.01471587417909036

373 6099
old [0.89776001] new 0.0022573153515691924

374 947
old [0.86356244] new 0.13362690423595175

375 2231
old [0.11111113] new 0.956402572430913

376 273
old [0.76546365] new 0.38808209455336934

377 2025
old [0.8836051] new 0.04243159836538937

378 20
old [0.53340034] new 0.4893584180890482

380 1136
old [0.89172781] new 0.054512396610899044

381 141
old [0.32782525] new 0.5198013684910917

382 310
old [0.22984155] new 0.6374052491987143

384 889
old [0.83423297] new 0.2307537432520888

385 467
old [0.73992509] new 0.426413200754921

386 23
old [0.58894155] new 0.5170513364562385

387 5938
old [0.0870009] new 0.9990488513093049

389 365
old [0.2

553 6277
old [0.92137256] new 0.0001990739999471079

554 136
old [0.67337219] new 0.4823174862403399

555 1829
old [0.85897795] new 0.07818450265850103

556 1126
old [0.12337519] new 0.9253971951921404

558 566
old [0.2491041] new 0.5659437555774706

559 1026
old [0.85625121] new 0.09394754102087863

560 26
old [0.44266992] new 0.48320359683291997

561 288
old [0.73938055] new 0.4320043004574703

563 182
old [0.27578468] new 0.5388715884085312

564 179
old [0.31401732] new 0.5465112168086356

565 1078
old [0.85606194] new 0.1240461598509649

566 427
old [0.71941185] new 0.468895743781861

567 112
old [0.64208286] new 0.5266740876219497

568 538
old [0.20165538] new 0.645655938100721

569 3563
old [0.09275665] new 0.9976959278199099

570 1629
old [0.11912043] new 0.9238433581406977

571 20
old [0.53884945] new 0.5306018270410356

572 171
old [0.67804098] new 0.4701367748283467

573 39
old [0.57150664] new 0.5021228356075603

574 308
old [0.70479146] new 0.42243822959878047

575 171
old 

742 348
old [0.22992892] new 0.634295385813992

743 2006
old [0.88070779] new 0.045592222873264206

744 172
old [0.30350462] new 0.5020357623892511

745 222
old [0.67793775] new 0.47758471307101635

747 363
old [0.29967979] new 0.5300424412817358

748 1092
old [0.12535945] new 0.9266892061441947

749 35
old [0.54997831] new 0.49997330129318746

750 75
old [0.37840582] new 0.4787287882236548

751 1251
old [0.13808411] new 0.8889934018638518

752 908
old [0.15333073] new 0.8043060463512908

754 700
old [0.78115236] new 0.3504920270389235

755 46
old [0.403098] new 0.48630844168869153

756 9
old [0.51229442] new 0.5007821701911683

757 407
old [0.74180193] new 0.44826629475746566

758 163
old [0.33680105] new 0.5429464920650645

759 164
old [0.64877126] new 0.5042300994907402

760 3500
old [0.08927296] new 0.9978833881846757

761 68
old [0.58988349] new 0.5170731925369305

762 177
old [0.6491527] new 0.4433928916174131

764 1189
old [0.89005488] new 0.05746803182490414

765 9
old [0.53949

928 654
old [0.18795042] new 0.6831453891537105

929 6608
old [0.06489971] new 0.9999688605579848

931 542
old [0.21175935] new 0.6734117899394003

932 87
old [0.39388814] new 0.5105997537913946

934 27
old [0.43937333] new 0.47461016924801147

935 177
old [0.6336764] new 0.5069160894852917

936 1519
old [0.14462441] new 0.8923249224923905

937 678
old [0.7895805] new 0.3561544911195129

938 542
old [0.21941061] new 0.6507203344899508

939 116
old [0.61916827] new 0.49859571412648557

940 59
old [0.41622631] new 0.49311957550685315

941 4345
old [0.07984637] new 0.9993488964823994

942 421
old [0.2415926] new 0.6120817011813794

943 178
old [0.66762754] new 0.5129057817882763

944 451
old [0.76042133] new 0.4137907663983034

945 76
old [0.61824119] new 0.5085247209407429

946 6147
old [0.93483725] new 1.7720616321203057e-05

947 34
old [0.44595794] new 0.48267218958386904

948 838
old [0.14788437] new 0.7665647817465462

949 183
old [0.64243834] new 0.5077129440848119

950 1
old [0.498

In [60]:
appro_ks= np.array(appro_ks)
new_predictions=np.array(new_predictions)
np.save("appro_ks_IP.npy", appro_ks)
np.save("new_predictions.npy", new_predictions)
np.save("old_predictions.npy", pred)

# Difference of change and prediction when k increasing

In [61]:
"""
row = len(list(range(0, X["dev"].shape[0])))
col = len(list(range(1, np.sum(y["train"]))))
changes = np.zeros((row, col))
prediction = np.zeros((row, col))

import warnings
warnings.filterwarnings("ignore")
for i in range(0, X["dev"].shape[0]):
    print(i)
    for j in range(1, np.sum(y["train"]):
        changes[i, j-1], _, prediction[i, j-1] = new_train(j, i, delta_pred)
        
   
np.save("pred_change_app.npy", prediction)
np.save("changes.npy", changes)

difference = prediction - changes
np.save("difference_k_appk.npy", difference)
"""

'\nrow = len(list(range(0, X["dev"].shape[0])))\ncol = len(list(range(1, np.sum(y["train"]))))\nchanges = np.zeros((row, col))\nprediction = np.zeros((row, col))\n\nimport warnings\nwarnings.filterwarnings("ignore")\nfor i in range(0, X["dev"].shape[0]):\n    print(i)\n    for j in range(1, np.sum(y["train"]):\n        changes[i, j-1], _, prediction[i, j-1] = new_train(j, i, delta_pred)\n        \n   \nnp.save("pred_change_app.npy", prediction)\nnp.save("changes.npy", changes)\n\ndifference = prediction - changes\nnp.save("difference_k_appk.npy", difference)\n'

In [44]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_tweet'